In [ ]:
import sys
sys.path.insert(1, './dlite-v2-1_5b/')
from instruct_pipeline import InstructionTextGenerationPipeline
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("dlite-v2-1_5b", padding_side="left")
model = AutoModelForCausalLM.from_pretrained("dlite-v2-1_5b", device_map="auto", torch_dtype=torch.bfloat16)

#model.to("cuda:0")

generate_text = InstructionTextGenerationPipeline(model=model, tokenizer=tokenizer)